# Interactive Pulumi Lab: Deploying a NixOS AMI on AWS

This notebook demonstrates how to use the Pulumi Automation API to interactively deploy a custom NixOS EC2 instance to AWS.

**Workflow:**
1.  **Setup:** Configure environment variables and import necessary libraries.
2.  **Define Infrastructure:** Create a Python function that defines the cloud resources (Security Group, EC2 Instance).
3.  **Run Pulumi:** Use the Automation API to create a Pulumi stack and deploy the resources.
4.  **View Outputs:** See the results of the deployment, like the instance ID.
5.  **Clean Up:** Destroy the resources after the experiment.


In [5]:
# 1. Setup and Imports
# This cell installs the required packages and imports necessary modules.

import os
import sys
import asyncio
import pulumi
from pathlib import Path
from pulumi.automation import LocalWorkspace, Stack, UpResult, ProjectSettings, ProjectBackend
from pulumi import automation as auto
from pulumi_aws import ec2
from pulumi_aws import Provider, ProviderEndpointArgs
from pulumi_aws.ec2 import get_security_group, SecurityGroupRule, Instance, DefaultSecurityGroup, DefaultSecurityGroupIngressArgs, DefaultSecurityGroupEgressArgs, KeyPair
from pulumi.automation._stack import StackInitMode

# Set the AWS region for the deployment
os.environ["AWS_DEFAULT_REGION"] = "eu-west-1"
# Use an empty passphrase for Pulumi, as we are not encrypting secrets in this example.
os.environ["PULUMI_CONFIG_PASSPHRASE"] = ""

# --- SSH Key Configuration ---
# Read the public key from the local filesystem.
# This assumes you have a key pair generated, and the public key is at ~/.ssh/aws.pub
try:
    ssh_public_key = Path("~/.ssh/aws.pub").expanduser().read_text()
    print("✅ SSH public key read successfully.")
except FileNotFoundError:
    print("❌ SSH public key not found at ~/.ssh/aws.pub. Please generate it first.")
    ssh_public_key = None # Set to None to handle the error gracefully


✅ SSH public key read successfully.


## 2. Define the Pulumi Infrastructure Program

This function contains the complete definition of our AWS resources. It's a standard Pulumi program, but wrapped in a Python function so we can pass it to the Automation API. It will:
- Create a provider pointing to LocalStack.
- Look up the default security group.
- Add an ingress rule to allow SSH.
- Launch an EC2 instance with the specified AMI.

In [6]:
def pulumi_program():
    """
    Defines the AWS resources for deploying the NixOS AMI on AWS.
    """
    # --- Configuration ---
    AMI_ID = "ami-098045a8abbd1fff1"
    INSTANCE_TYPE = "t3.small"
    CUSTOM_SSH_PORT = 22
    KEY_NAME = "aws-key-pulumi" # A name for the key pair in AWS

    # --- Create a Key Pair in AWS from the local public key ---
    if ssh_public_key:
        aws_key_pair = ec2.KeyPair(
            KEY_NAME,
            key_name=KEY_NAME,
            public_key=ssh_public_key,
        )
        key_name_param = aws_key_pair.key_name
    else:
        # If the key file was not found, we proceed without a key.
        # This will likely make SSH access impossible unless using other auth methods.
        print("⚠️  Proceeding without an SSH key. You may not be able to connect to the instance.")
        key_name_param = None


    # --- Get the Default VPC to find the Default Security Group ---
    default_vpc = ec2.get_vpc(default=True)

    # --- Take Authoritative Control of the 'default' Security Group ---
    managed_default_sg = ec2.DefaultSecurityGroup(
        "manage-default-sg",
        vpc_id=default_vpc.id,
        ingress=[
            ec2.DefaultSecurityGroupIngressArgs(
                protocol="tcp",
                from_port=CUSTOM_SSH_PORT,
                to_port=CUSTOM_SSH_PORT,
                cidr_blocks=["0.0.0.0/0"],
                description="Allow custom SSH access",
            )
        ],
        egress=[
            ec2.DefaultSecurityGroupEgressArgs(
                protocol="-1", from_port=0, to_port=0, cidr_blocks=["0.0.0.0/0"]
            )
        ],
    )

    # --- EC2 Instance Definition ---
    nixos_instance = Instance(
        "nixos-aws-instance",
        ami=AMI_ID,
        instance_type=INSTANCE_TYPE,
        key_name=key_name_param, # Assign the created key pair
        vpc_security_group_ids=[managed_default_sg.id],
    )

    # --- Outputs ---
    pulumi.export("instance_id", nixos_instance.id)
    pulumi.export("instance_public_ip", nixos_instance.public_ip)
    pulumi.export("instance_private_ip", nixos_instance.private_ip)

print("Pulumi program defined successfully.")


Pulumi program defined successfully.


## 3. Deploy the Infrastructure

This cell uses the Pulumi Automation API to execute the `pulumi_program`. It performs the following actions:
- Creates a `LocalWorkspace` to manage our project.
- Selects or creates a stack (e.g., `localstack-dev`).
- Sets the AWS region configuration for the stack.
- Runs `stack.up()` which is equivalent to `pulumi up` on the command line.
- Prints the outputs from the deployment.

In [7]:
# --- Automation API Execution ---
stack_name = "aws-dev"
project_name = "aws-nixos-deployment"

!mkdir -p /tmp/.pulumi/stacks
# Define the project settings programmatically.
project_settings = ProjectSettings(
    name=project_name,
    runtime="python",
    backend=ProjectBackend("file:///tmp/.pulumi/stacks")
)

# Create a workspace with the defined project settings.
ws = auto.LocalWorkspace(
    project_settings=project_settings
)

# Select the stack, or create it if it does not exist.
try:
    stack = ws.select_stack(stack_name)
    print(f"Stack '{stack_name}' successfully selected.")
except Exception:
    print(f"Stack '{stack_name}' not found, creating it...")
    stack = ws.create_stack(stack_name)
    print(f"Stack '{stack_name}' created.")
    stack = ws.select_stack(stack_name)

ws.program = pulumi_program

st = auto.Stack(stack_name, ws, StackInitMode.CREATE_OR_SELECT)

# Clear any existing locks before proceeding
try:
    print("🔓 Clearing any existing locks...")
    st.cancel()
    print("✅ Lock cleared.")
    # Run the deployment
    up_res = st.up(on_output=print)
    print("✅ Deployment successful!")
    
    # Print outputs
    for key, value in up_res.outputs.items():
        print(f"{key}: {value.value}")

except Exception as e:
    print(f"ℹ️  An error occurred: {e}")
    # In case of an error, it's good practice to try to cancel any lingering lock.
    try:
        st.cancel()
        print("✅ Lock cleared after error.")
    except Exception as cancel_e:
        print(f"ℹ️  Could not clear lock after error: {cancel_e}")


I0000 00:00:1759754459.649765 1359166 fork_posix.cc:71] Other threads are currently calling into gRPC, skipping fork() handlers


Stack 'aws-dev' successfully selected.
🔓 Clearing any existing locks...
✅ Lock cleared.
Updating (aws-dev):

@ updating....
    pulumi:pulumi:Stack aws-nixos-deployment-aws-dev running
    aws:ec2:KeyPair aws-key-pulumi
    aws:ec2:DefaultSecurityGroup manage-default-sg
 ++ aws:ec2:Instance nixos-aws-instance creating replacement (0s) [diff: ~ami,instanceType,publicDns,publicIp]
@ updating................
 ++ aws:ec2:Instance nixos-aws-instance created replacement (12s) [diff: ~ami,instanceType,publicDns,publicIp]
 +- aws:ec2:Instance nixos-aws-instance replacing (0s) [diff: ~ami,instanceType,publicDns,publicIp]
 +- aws:ec2:Instance nixos-aws-instance replaced (0.00s) [diff: ~ami,instanceType,publicDns,publicIp]
 -- aws:ec2:Instance nixos-aws-instance deleting original (0s) [diff: ~ami,instanceType,publicDns,publicIp]
 -- aws:ec2:Instance nixos-aws-instance deleted original (0.19s) [diff: ~ami,instanceType,publicDns,publicIp]
    pulumi:pulumi:Stack aws-nixos-deployment-aws-dev
Outputs

## 4. Clean Up Resources

It's important to destroy the resources you've created to avoid leaving orphaned containers running. This cell runs `stack.destroy()` to tear down everything that was created in the previous step.

In [8]:
print("Destroying resources...")
try:
    stack.destroy(on_output=print)
    print("\nCleanup successful!")
except Exception as e:
    print(f"\nAn error occurred during cleanup: {e}")

Destroying resources...

An error occurred during cleanup: 'NoneType' object has no attribute 'destroy'
